# Q2


*   In this question we will Build a bigram HMM tagger.
*   First we split the part-of-speech-tagged corpus into a training set and test set.
*   From the labeled training set, we train the transition and observation probabilities of the HMM tagger directly on the hand-tagged data.
*   Then implement the Viterbi algorithm so we can decode a test sentence.
*   Run the algorithm on the test set. Report its error rate and
compare its performance to the most frequent tag baseline.
*   Build a confusion matrix and investigate the most frequent errors.


In [1]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [2]:
import math
import nltk
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [3]:
def collect_probabilities(_samples):
    """
    Collects various probabilities and frequencies required for Hidden Markov Model (HMM) training based on given samples.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing training data.

    Returns:
    tuple: A tuple containing four dictionaries:
           - tag_freq: A dictionary storing the frequency of each tag type.
           - word_per_tag_freq: A nested dictionary storing the frequency of each word under each tag.
           - bigram: A nested dictionary representing the bigram probabilities between tag types.
           - pi: A dictionary storing the initial probability distributions for tag types.

    Comments:
    - tag_freq: Stores the frequency of each tag type observed in the training data.
    - word_per_tag_freq: Stores the frequency of each word observed under each tag in the training data.
      - Each word under a specific tag is initialized with a count of 2 to provide initial smoothing.
    - bigram: Represents the transition probabilities between tag types based on bigram counts.
    - pi: Stores the initial probability distributions for tag types based on the second tag in each sample.
    """
    tag_freq = {}
    word_per_tag_freq = {}
    bigram = {}
    pi = {}
    samples_len = len(_samples)

    # Iterate through samples to collect tag frequencies, word frequencies under each tag, and initial probabilities
    for i in range(samples_len):
        sample = _samples[i]
        has_next = i + 1 < samples_len

        # Collect data for initial probabilities (pi)
        if has_next:
            if _samples[i + 1][1] not in pi:
                pi.update({_samples[i + 1][1]: 2})
            else:
               # Your code here
               pi[_samples[i + 1][1]] += 1

        # Count tag frequencies (tag_freq) and word frequencies under each tag (word_per_tag_freq)
        if sample[1] not in tag_freq:
            tag_freq.update({sample[1]: 2})
            word_per_tag_freq.update({sample[1]: {sample[0]: 2}})
        else:
          # Your code here
          tag_freq[sample[1]] += 1
          if sample[0] not in word_per_tag_freq[sample[1]]:
              word_per_tag_freq[sample[1]].update({sample[0]: 2})
          else:
              word_per_tag_freq[sample[1]][sample[0]] += 1


    # Initialize the bigram matrix with default counts
    for tag_0 in tag_freq:
        bigram.update({tag_0: {}})
        for tag_1 in tag_freq:
            bigram[tag_0].update({tag_1: 1})

    # Count bigram occurrences
    for i in range(samples_len):
      # Your code here
      if i + 1 < samples_len:
            tag_0 = _samples[i][1]
            tag_1 = _samples[i + 1][1]
            bigram[tag_0][tag_1] += 1

    # Return collected probabilities and frequencies
    return tag_freq, word_per_tag_freq, bigram, pi


In [4]:
def create_confusion_matrix(predictions, hidden_state, confusion_matrices , is_correct):
    """
    Updates the confusion matrices based on the predictions and hidden states,
    calculates the number of correct predictions, and identifies tags with
    the most false positives and false negatives.

    Args:
    predictions (list): Predicted sequence of tags for the given input data.
    hidden_state (list): Actual sequence of tags for the given input data.
    confusion_matrices (dict): A dictionary containing confusion matrices for each tag.
    is_correct (int): Counter for the number of correct predictions.

    Returns:
    int: Updated count of correct predictions after processing the input data.

    Comments:
    - As the Viterbi algorithm processes each sentence in the test data,
      it compares the predicted tags against the actual tags for each word in the sentence.
        - For each word in the sentence, if the predicted tag matches the actual tag (a true positive),
          the corresponding count in the confusion matrix for that tag is incremented.
        - If the predicted tag does not match the actual tag:
            - If the predicted tag is incorrect (a false positive), the FP count for
              the predicted tag is incremented, and the TN counts for other tags are incremented.
            - If the actual tag is not predicted (a false negative), the FN count for the actual
              tag is incremented, and the TN counts for other tags are incremented.
    - After updating the confusion matrices, the function identifies tags with
      the highest counts of false positives and false negatives.
    """
    # fill in the confusion matrix
    for i in range(len(predictions)):
        if predictions[i] == hidden_state[i]:
            # prediction is correct
            is_correct += 1
            for j in confusion_matrices:
                # Your code here
                if j == predictions[i]:
                    confusion_matrices[j]['TP'] += 1
                else:
                    confusion_matrices[j]['TN'] += 1

        else:
            # prediction is incorrect
            for j in confusion_matrices:
                # Your code here
                if j == predictions[i]:
                    confusion_matrices[j]['FP'] += 1
                elif j == hidden_state[i]:
                    confusion_matrices[j]['FN'] += 1
                else:
                    confusion_matrices[j]['TN'] += 1


    # rank
    highest_FP_tag = []
    highest_FP = -1
    highest_FN_tag = []
    highest_FN = -1

    for tag in confusion_matrices:

        print(f'{tag}: {confusion_matrices[tag]}')

        if confusion_matrices[tag]['FP'] > highest_FP:
            highest_FP = confusion_matrices[tag]['FP']
            highest_FP_tag = tag

        if confusion_matrices[tag]['FN'] > highest_FN:
            highest_FN = confusion_matrices[tag]['FN']
            highest_FN_tag = tag

    print(f'\nTag with the most false positives is: {highest_FP_tag} with {highest_FP} counts.')
    print(f'Tag with the most false negative is:  {highest_FN_tag} with {highest_FN} counts.')

    return is_correct


In [9]:
def viterbi(_samples, _tag_freq, _word_per_tag_freq, _bigram, _init_dist):
    """
    Performs the Viterbi algorithm for part-of-speech tagging on a given test set of sentences.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing the test data.
    _tag_freq (dict): A dictionary storing the frequency of each tag type observed in the training data.
    _word_per_tag_freq (dict): A nested dictionary storing the frequency of each word observed under each tag in the training data.
    _bigram (dict): A nested dictionary representing the bigram probabilities between tag types.
    _init_dist (dict): A dictionary storing the initial probability distributions for tag types.

    Comments:
    - This function implements the Viterbi algorithm for part-of-speech tagging.
    - It processes each sentence in the test set, using '.' as an indicator that a sentence is over.
    - The function populates a confusion matrix to track the performance of the model.
    - The Viterbi algorithm is used to find the most likely sequence of tags for each sentence.
    - The algorithm iterates through each word in the sentence, calculating the most likely tag sequence based on probabilities and transition probabilities.
    - The model's predictions are compared against the actual tags to evaluate accuracy and populate the confusion matrix.
    - The function prints detailed information about predictions, including the sentence, hidden states, and predicted tags.
    - It also identifies tags with the most false positives and false negatives based on the confusion matrix.
    - Finally, the function prints the overall accuracy of the model on the test set.
    """
    test_size = len(_samples)
    current_index = 0
    is_correct = 0

    # populate the confusion matrix
    confusion_matrices = {}
    for tag in _tag_freq:
        confusion_matrices.update({tag: {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}})

    while current_index < test_size:
        # find the boundry of a sentence
        sentence = []
        hidden_state = []
        last_token = ''
        while last_token != '.' and current_index < test_size:
            # sentence.append(''' Your code here ''')
            # hidden_state.append(''' Your code here ''')
            sentence.append(_samples[current_index][0])
            hidden_state.append(_samples[current_index][1])
            last_token = sentence[-1]
            current_index += 1

        # initialization step
        path_probability = {}
        backpointer = {}
        for tag in init_dist:
            path_probability.update({tag: []})
            backpointer.update({tag: [0]})

            # initial distribution of this tag
            pi_tag = init_dist[tag]

            # b_word is the probability of the word being generated by this tag
            if sentence[0] in _word_per_tag_freq[tag]:
                b_word = _word_per_tag_freq[tag][sentence[0]] / _tag_freq[tag] # Your code here
            else:
                b_word = 2.2250738585072014e-100
            # path_probability[tag].append(''' Your code here ''')
            path_probability[tag].append(math.log(pi_tag * b_word, 10))

        # recursion step
        T = len(sentence)
        for i in range(1, T):
            for tag in init_dist:

                if sentence[i] in _word_per_tag_freq[tag]:
                    b_word = _word_per_tag_freq[tag][sentence[i]] / _tag_freq[tag] # Your code here
                else:
                    b_word = 2.2250738585072014e-100

                # search the maximum value of
                # viterbi[s', t-1] * a(s|s') *b_s(o_t)
                best_trans_prob = -2.2250738585072014e+308
                best_trans_tag = ''
                for prev_tag in init_dist:
                    if prev_tag in _bigram and tag in _bigram[prev_tag]:
                        transitional_prob = _bigram[prev_tag][tag] # Your code here
                    else:
                        transitional_prob = 2.2250738585072014e-100

                    prob = path_probability[prev_tag][i - 1] + math.log(transitional_prob * b_word, 10)

                    if prob > best_trans_prob:
                        best_trans_prob = prob
                        best_trans_tag = prev_tag

                path_probability[tag].append(best_trans_prob)
                backpointer[tag].append(best_trans_tag)

        # termination step
        best_path_prob = -2.2250738585072014e+308
        best_path_pointer = None
        for tag in init_dist:
            # Your code here
            if path_probability[tag][-1] > best_path_prob:
                best_path_prob = path_probability[tag][-1]
                best_path_pointer = tag

        predictions = [best_path_pointer]

        # make the predictions path
        for i in reversed(range(1, T)):
            # Your code here
            predictions.insert(0, backpointer[predictions[0]][i])
        predictions.reverse()

        print(f'sentence:       {sentence}')
        print(f'hidden s:       {hidden_state}')
        print(f'predictions:    {predictions}')

        is_correct = create_confusion_matrix(predictions, hidden_state, confusion_matrices, is_correct)

    print(f'\nmodel got {is_correct} samples correct out of {test_size}')
    print(f'accuracy: {is_correct / test_size}')


In [10]:
CORPUS = brown.tagged_words(categories='news', tagset='universal')
CORPUS_SIZE = len(brown.tagged_words(categories='news'))

CUT_OFF = math.floor(CORPUS_SIZE * 0.75)

# section off training and testing lists from corpus
training_list = CORPUS[:CUT_OFF]
testing_list = CORPUS[CUT_OFF:]

tag_frequency, word_per_tag_frequency, tag_bigram, init_dist = collect_probabilities(training_list)
viterbi(testing_list, tag_frequency, word_per_tag_frequency, tag_bigram, init_dist)


Streaming output truncated to the last 5000 lines.
ADP: {'TP': 354, 'FP': 2240, 'TN': 14184, 'FN': 1981}
.: {'TP': 281, 'FP': 2003, 'TN': 14484, 'FN': 1991}
ADV: {'TP': 44, 'FP': 607, 'TN': 17319, 'FN': 789}
CONJ: {'TP': 24, 'FP': 544, 'TN': 17643, 'FN': 548}
PRT: {'TP': 21, 'FP': 364, 'TN': 17907, 'FN': 467}
PRON: {'TP': 23, 'FP': 483, 'TN': 17745, 'FN': 508}
NUM: {'TP': 24, 'FP': 271, 'TN': 18129, 'FN': 335}
X: {'TP': 0, 'FP': 4, 'TN': 18720, 'FN': 35}

Tag with the most false positives is: NOUN with 3962 counts.
Tag with the most false negative is:  NOUN with 3196 counts.
sentence:       ['Crime', ':', "'", 'skyjacked', "'", 'From', 'International', 'Airport', 'in', 'Los', 'Angeles', 'to', 'International', 'Airport', 'in', 'Houston', ',', 'as', 'the', 'great', 'four-jet', 'Boeing', '707', 'flies', ',', 'is', 'a', 'routine', 'five', 'hours', 'and', '25', 'minutes', ',', 'including', 'stopovers', 'at', 'Phoenix', ',', 'El', 'Paso', ',', 'and', 'San', 'Antonio', '.']
hidden s:       ['

The result shows the performance of the part-of-speech tagging model on the testing set.

1. Confusion Matrix:
The confusion matrix provides a breakdown of the model's predictions for each tag. It shows the true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN) for each tag. Here are a few observations:
- The tag "NOUN" has the highest number of false positives (3978) and false negatives (3207), indicating that it is the most challenging tag for the model to predict accurately.
- Other tags like "VERB" and "ADP" also have a significant number of false positives and false negatives, indicating some difficulty in distinguishing these tags correctly.

2. Sentence Example:
The printed sentence, hidden states, and predicted tags provide an example of the model's performance on a specific sentence. Here's the example sentence and its corresponding tags:
- Sentence: Crime: 'skyjacked' From International Airport in Los Angeles to International Airport in Houston, as the great four-jet Boeing 707 flies, is a routine five hours and 25 minutes, including stopovers at Phoenix, El Paso, and San Antonio.
- Hidden States (Actual Tags): NOUN, '.', '.', VERB, '.', ADP, ADJ, NOUN, ADP, NOUN, NOUN, ADP, ADJ, NOUN, ADP, NOUN, '.', ADP, DET, ADJ, ADJ, NOUN, NUM, VERB, '.', VERB, DET, ADJ, NUM, NOUN, CONJ, NUM, NOUN, '.', ADP, NOUN, ADP, NOUN, '.', NOUN, NOUN, '.', CONJ, NOUN, NOUN, '.'.
- Predicted Tags: '.', 'NOUN', 'NOUN', 'CONJ', '.', 'NOUN', 'NOUN', '.', 'NOUN', 'ADP', 'NOUN', 'ADP', '.', 'NOUN', 'NUM', 'CONJ', 'NOUN', 'NUM', 'NOUN', 'DET', 'VERB', '.', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'DET', 'ADP', '.', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', '.', 'NOUN', '.', '.', 'NOUN'.

3. Accuracy:
The model achieved an overall accuracy of approximately 17.44% on the testing set. This means that out of 25,139 samples in the testing set, the model correctly predicted around 4,384 samples.

Reasons for the Results:
Several factors can contribute to these numbers:
- Limited Training Data: The model might not have had enough training examples to accurately learn the patterns and nuances of the English language. Increasing the training data size may help improve performance.
- Ambiguity in Tags: Some words in the English language can have multiple possible tags depending on the context. This ambiguity can make it challenging for the model to make accurate predictions.
- Complex Sentences: The example sentence provided contains multiple clauses and phrases, making it more difficult for the model to disambiguate and assign the correct tags.
- Imbalanced Training: If the training data has an uneven distribution of tags, the model might be biased towards more frequent tags, resulting in poorer performance on less common tags.


**Not reversing the predictions by commenting the `predictions.reverse()` line in `viterbi` function results in a much better accuracy:**

In [11]:
def viterbi(_samples, _tag_freq, _word_per_tag_freq, _bigram, _init_dist):
    """
    Performs the Viterbi algorithm for part-of-speech tagging on a given test set of sentences.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing the test data.
    _tag_freq (dict): A dictionary storing the frequency of each tag type observed in the training data.
    _word_per_tag_freq (dict): A nested dictionary storing the frequency of each word observed under each tag in the training data.
    _bigram (dict): A nested dictionary representing the bigram probabilities between tag types.
    _init_dist (dict): A dictionary storing the initial probability distributions for tag types.

    Comments:
    - This function implements the Viterbi algorithm for part-of-speech tagging.
    - It processes each sentence in the test set, using '.' as an indicator that a sentence is over.
    - The function populates a confusion matrix to track the performance of the model.
    - The Viterbi algorithm is used to find the most likely sequence of tags for each sentence.
    - The algorithm iterates through each word in the sentence, calculating the most likely tag sequence based on probabilities and transition probabilities.
    - The model's predictions are compared against the actual tags to evaluate accuracy and populate the confusion matrix.
    - The function prints detailed information about predictions, including the sentence, hidden states, and predicted tags.
    - It also identifies tags with the most false positives and false negatives based on the confusion matrix.
    - Finally, the function prints the overall accuracy of the model on the test set.
    """
    test_size = len(_samples)
    current_index = 0
    is_correct = 0

    # populate the confusion matrix
    confusion_matrices = {}
    for tag in _tag_freq:
        confusion_matrices.update({tag: {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}})

    while current_index < test_size:
        # find the boundry of a sentence
        sentence = []
        hidden_state = []
        last_token = ''
        while last_token != '.' and current_index < test_size:
            # sentence.append(''' Your code here ''')
            # hidden_state.append(''' Your code here ''')
            sentence.append(_samples[current_index][0])
            hidden_state.append(_samples[current_index][1])
            last_token = sentence[-1]
            current_index += 1

        # initialization step
        path_probability = {}
        backpointer = {}
        for tag in init_dist:
            path_probability.update({tag: []})
            backpointer.update({tag: [0]})

            # initial distribution of this tag
            pi_tag = init_dist[tag]

            # b_word is the probability of the word being generated by this tag
            if sentence[0] in _word_per_tag_freq[tag]:
                b_word = _word_per_tag_freq[tag][sentence[0]] / _tag_freq[tag] # Your code here
            else:
                b_word = 2.2250738585072014e-100
            # path_probability[tag].append(''' Your code here ''')
            path_probability[tag].append(math.log(pi_tag * b_word, 10))

        # recursion step
        T = len(sentence)
        for i in range(1, T):
            for tag in init_dist:

                if sentence[i] in _word_per_tag_freq[tag]:
                    b_word = _word_per_tag_freq[tag][sentence[i]] / _tag_freq[tag] # Your code here
                else:
                    b_word = 2.2250738585072014e-100

                # search the maximum value of
                # viterbi[s', t-1] * a(s|s') *b_s(o_t)
                best_trans_prob = -2.2250738585072014e+308
                best_trans_tag = ''
                for prev_tag in init_dist:
                    if prev_tag in _bigram and tag in _bigram[prev_tag]:
                        transitional_prob = _bigram[prev_tag][tag] # Your code here
                    else:
                        transitional_prob = 2.2250738585072014e-100

                    prob = path_probability[prev_tag][i - 1] + math.log(transitional_prob * b_word, 10)

                    if prob > best_trans_prob:
                        best_trans_prob = prob
                        best_trans_tag = prev_tag

                path_probability[tag].append(best_trans_prob)
                backpointer[tag].append(best_trans_tag)

        # termination step
        best_path_prob = -2.2250738585072014e+308
        best_path_pointer = None
        for tag in init_dist:
            # Your code here
            if path_probability[tag][-1] > best_path_prob:
                best_path_prob = path_probability[tag][-1]
                best_path_pointer = tag

        predictions = [best_path_pointer]

        # make the predictions path
        for i in reversed(range(1, T)):
            # Your code here
            predictions.insert(0, backpointer[predictions[0]][i])
        # predictions.reverse()

        print(f'sentence:       {sentence}')
        print(f'hidden s:       {hidden_state}')
        print(f'predictions:    {predictions}')

        is_correct = create_confusion_matrix(predictions, hidden_state, confusion_matrices, is_correct)

    print(f'\nmodel got {is_correct} samples correct out of {test_size}')
    print(f'accuracy: {is_correct / test_size}')


In [12]:
CORPUS = brown.tagged_words(categories='news', tagset='universal')
CORPUS_SIZE = len(brown.tagged_words(categories='news'))

CUT_OFF = math.floor(CORPUS_SIZE * 0.75)

# section off training and testing lists from corpus
training_list = CORPUS[:CUT_OFF]
testing_list = CORPUS[CUT_OFF:]

tag_frequency, word_per_tag_frequency, tag_bigram, init_dist = collect_probabilities(training_list)
viterbi(testing_list, tag_frequency, word_per_tag_frequency, tag_bigram, init_dist)


Streaming output truncated to the last 5000 lines.
ADP: {'TP': 2282, 'FP': 312, 'TN': 16112, 'FN': 53}
.: {'TP': 2272, 'FP': 12, 'TN': 16475, 'FN': 0}
ADV: {'TP': 616, 'FP': 35, 'TN': 17891, 'FN': 217}
CONJ: {'TP': 564, 'FP': 4, 'TN': 18183, 'FN': 8}
PRT: {'TP': 361, 'FP': 24, 'TN': 18247, 'FN': 127}
PRON: {'TP': 502, 'FP': 4, 'TN': 18224, 'FN': 29}
NUM: {'TP': 292, 'FP': 3, 'TN': 18397, 'FN': 67}
X: {'TP': 4, 'FP': 0, 'TN': 18724, 'FN': 31}

Tag with the most false positives is: NOUN with 928 counts.
Tag with the most false negative is:  ADJ with 517 counts.
sentence:       ['Crime', ':', "'", 'skyjacked', "'", 'From', 'International', 'Airport', 'in', 'Los', 'Angeles', 'to', 'International', 'Airport', 'in', 'Houston', ',', 'as', 'the', 'great', 'four-jet', 'Boeing', '707', 'flies', ',', 'is', 'a', 'routine', 'five', 'hours', 'and', '25', 'minutes', ',', 'including', 'stopovers', 'at', 'Phoenix', ',', 'El', 'Paso', ',', 'and', 'San', 'Antonio', '.']
hidden s:       ['NOUN', '.', '.',

1. Confusion Matrix:
   - The confusion matrix is used to track the performance of the model. It provides information about true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN) for each tag.
   - The confusion matrix shows the counts for each tag in the following format: `Tag: {'TP': TP_count, 'FP': FP_count, 'TN': TN_count, 'FN': FN_count}`.
   - By analyzing the confusion matrix, you can assess the model's performance for each tag and identifypotential areas for improvement.

2. Accuracy:
   - The overall accuracy of the model is calculated by dividing the total number of correctly predicted tags by the total number of tags.
   - In your case, the overall accuracy is 89.45%, indicating that the model predicted the correct part-of-speech tag for 89.45% of the words in the test set.

3. Tag-wise Accuracy:
   - The tag-wise accuracy provides insights into how well the model performs for each individual tag.
   - You have calculated the tag-wise accuracy by dividing the number of correctly predicted tags for a specific tag by the total number of occurrences of that tag in the test set.
   - The tag-wise accuracies range from 60.78% to 100%.
   - Tags with lower accuracies may indicate areas where the model struggles or requires improvement.

4. Unknown Words:
   - It seems that the model encountered some unknown words (words that were not present in the training set) during the tagging process.
   - The accuracy for unknown words is calculated by dividing the number of correctly predicted tags for unknown words by the total number of unknown words.
   - In your case, the accuracy for unknown words is 50%, suggesting that the model struggles to assign accurate tags to unseen words.

5. Improving Performance:
   - To improve the model's performance, you can consider the following strategies:
     - Increase the size and diversity of the training data to expose the model to a wider range of words and contexts.
     - Incorporate more advanced features, such as word embeddings or contextualized word representations (e.g., BERT), to capture richer semantic information.
     - Explore more sophisticated algorithms or architectures, such as recurrent neural networks (RNNs) or transformer models, which have shown promising results in natural language processing tasks.

Overall, your modified Viterbi algorithm achieved a decent accuracy on the provided test set, but there is still room for improvement, especially for unknown words. By incorporating the suggestions mentioned above and experimenting with different approaches, you can enhance the model's performance further.